# AI-Powered Photo Editing: Comprehensive Guide to Intelligent Image Enhancement

## Overview

This notebook combines three key aspects of our AI-powered photo editing system:

1. **AI-Powered Image Analysis**: Automatically analyzing image content and recommending adjustments
2. **Natural Language Photo Editing**: Editing photos using plain English instructions
3. **RAG-Based Style Recommendations**: Suggesting cinematic styles based on image content

## Problem Statement

Photo editing traditionally requires significant technical knowledge and artistic skill. Users need to understand complex concepts like exposure, contrast, color balance, and composition to achieve professional-looking results. This creates a high barrier to entry for casual photographers who want to enhance their images but lack the technical expertise.

Additionally, the editing process can be time-consuming, requiring multiple adjustments and trial-and-error to achieve the desired look. This is especially challenging when dealing with different types of images (landscapes, portraits, low-light scenes) that each require specialized editing approaches.

Traditional photo editing interfaces rely on technical sliders, complex terminology, and specialized knowledge that can be intimidating for casual users. Even with simplified consumer apps, users must:

1. Understand what each control does (exposure, contrast, saturation, etc.)
2. Know which adjustments to make for specific visual goals
3. Make multiple trial-and-error attempts to achieve desired results
4. Remember which combinations of settings create specific looks

Creating professional-looking, cinematic styles for photos traditionally requires:

1. **Extensive knowledge of cinematography** and color grading techniques
2. **Understanding of visual aesthetics** from different film genres and directors
3. **Technical expertise** in complex editing software
4. **Time-consuming experimentation** to achieve desired looks

## How Generative AI Solves These Problems

Generative AI transforms the photo editing experience by bringing intelligence and automation to the process. Instead of requiring users to understand technical details, AI can:

1. **Analyze image content** to understand what's in the photo (people, landscapes, objects)
2. **Assess technical qualities** like lighting conditions, color balance, and exposure
3. **Recommend appropriate adjustments** based on the specific content and conditions
4. **Understand natural language instructions** from users who can describe what they want in plain English
5. **Suggest cinematic styles** that match the image content using knowledge of cinematography techniques

Natural Language Processing (NLP) combined with computer vision creates a revolutionary approach to photo editing. Instead of manipulating technical controls, users can simply describe what they want in plain English:

- "Make the sunset colors more vibrant"
- "Add more contrast and warmth to the portrait"
- "Give this landscape a dramatic cinematic look"
- "Fix the lighting in this dark indoor photo"

Retrieval Augmented Generation (RAG) combined with computer vision creates an intelligent style recommendation system that:

1. **Analyzes image content** to understand the scene type, lighting conditions, and subjects
2. **Retrieves knowledge** about cinematography techniques and styles from a curated database
3. **Matches content to appropriate styles** based on cinematography principles
4. **Explains recommendations** with clear reasoning about why each style works for the image
5. **Allows natural language queries** so users can describe the look they want to achieve

This notebook demonstrates how our AI Photo Editor uses these capabilities to make professional-quality editing accessible to everyone, regardless of technical expertise.


## Setup and Imports

Let's start by importing the necessary libraries and setting up our environment.


In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import json
import logging
import torch
import requests
import io
from typing import Dict, List, Any, Tuple, Optional, Callable, Union, BinaryIO

# Set up matplotlib for displaying images
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define supported image formats
SUPPORTED_FORMATS = {
    # Standard formats
    '.jpg': 'JPEG',
    '.jpeg': 'JPEG',
    '.png': 'PNG',
    '.tiff': 'TIFF',
    '.tif': 'TIFF',
    '.bmp': 'BMP',
    '.gif': 'GIF',
}

# Implementation of utility functions
def load_image(source: Union[str, BinaryIO, np.ndarray]) -> np.ndarray:
    """Load an image from a file path, URL, file object, or numpy array.

    Args:
        source: Source image as a file path, URL, file object, or numpy array

    Returns:
        Loaded image as numpy array in RGB format

    Raises:
        ValueError: If image could not be loaded
    """
    # If source is already a numpy array, just return it
    if isinstance(source, np.ndarray):
        return source

    # If source is a file path or URL, load from path or download from URL
    if isinstance(source, str):
        # Check if source is a URL
        if source.startswith('http://') or source.startswith('https://'):
            try:
                # Download the image from the URL
                response = requests.get(source, timeout=10)
                response.raise_for_status()  # Raise an exception for HTTP errors

                # Create a file-like object from the response content
                image_data = io.BytesIO(response.content)

                # Open the image using PIL
                with Image.open(image_data) as img:
                    # Convert to RGB if needed
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                    return np.array(img)
            except Exception as url_error:
                raise ValueError(f"Failed to load image from URL {source}: {str(url_error)}")
        else:
            # Handle local file path
            try:
                with Image.open(source) as img:
                    # Convert to RGB if needed
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                    return np.array(img)
            except Exception as pil_error:
                # Fall back to OpenCV
                try:
                    image = cv2.imread(source)
                    if image is None:
                        raise ValueError(f"Could not load image at {source}")
                    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                except Exception as cv_error:
                    raise ValueError(f"Failed to load image with both PIL and OpenCV: {str(cv_error)}")

    # If source is a file-like object, load from file object
    try:
        # Save the current position
        pos = source.tell()

        # Reset to beginning
        source.seek(0)

        with Image.open(source) as img:
            if img.mode != 'RGB':
                img = img.convert('RGB')
            image_array = np.array(img)

        # Reset to original position
        source.seek(pos)
        return image_array
    except Exception as pil_error:
        # Fall back to OpenCV
        try:
            # Reset to beginning
            source.seek(0)
            file_bytes = np.asarray(bytearray(source.read()), dtype=np.uint8)
            image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
            if image is None:
                raise ValueError(f"Could not decode image from file object")
            # Reset to original position
            source.seek(pos)
            return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as cv_error:
            # Reset position before raising
            source.seek(pos)
            raise ValueError(f"Failed to load image with both PIL and OpenCV: {str(cv_error)}")

def save_image(image: np.ndarray, output_path: str, quality: int = 95) -> None:
    """Save an image to file path.

    Args:
        image: Image as numpy array in RGB format
        output_path: Path where the image will be saved
        quality: Quality for lossy formats (0-100)

    Raises:
        RuntimeError: If image saving fails
    """
    try:
        # Create directory if it doesn't exist
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir, exist_ok=True)

        # Convert numpy array to PIL Image
        pil_image = Image.fromarray(image.astype('uint8'), 'RGB')

        # Determine format from file extension
        ext = os.path.splitext(output_path)[1].lower()
        format_name = SUPPORTED_FORMATS.get(ext, 'JPEG')

        # Save with appropriate parameters
        save_args = {}
        if format_name == 'JPEG':
            save_args['quality'] = quality
            save_args['optimize'] = True
        elif format_name == 'PNG':
            save_args['optimize'] = True

        pil_image.save(output_path, format=format_name, **save_args)
    except Exception as e:
        raise RuntimeError(f"Error saving image to {output_path}: {str(e)}")

def normalize_image(image: np.ndarray) -> np.ndarray:
    """Normalize image values to [0, 1] range."""
    return image.astype(np.float32) / 255.0

def denormalize_image(image: np.ndarray) -> np.ndarray:
    """Convert normalized image back to [0, 255] range."""
    return (image * 255).astype(np.uint8)

# Define Adjustment class
class Adjustment:
    """Represents an image adjustment parameter."""

    def __init__(self, parameter: str, suggested: float, unit: str = "", description: str = ""):
        """Initialize an adjustment.

        Args:
            parameter: Name of the parameter to adjust
            suggested: Suggested value for the adjustment
            unit: Unit of measurement (e.g., "EV", "percent")
            description: Human-readable description of the adjustment
        """
        self.parameter = parameter
        self.suggested = suggested
        self.unit = unit
        self.description = description

    def __repr__(self) -> str:
        """String representation of the adjustment."""
        return f"Adjustment({self.parameter}, {self.suggested} {self.unit}, '{self.description}')"

# Implementation of ImageAnalyzer class
class ImageAnalyzer:
    """Analyzes images and recommends adjustments."""

    def __init__(self):
        """Initialize the image analyzer."""
        pass

    def analyze(self, image_source: Union[str, np.ndarray]) -> List[Adjustment]:
        """Analyze an image and recommend adjustments.

        Args:
            image_source: Path to the image or a numpy array

        Returns:
            List of recommended adjustments
        """
        # Load the image if it's a file path
        image = load_image(image_source)

        # Analyze the image
        adjustments = []

        # Check exposure
        exposure_adjustment = self._analyze_exposure(image)
        if exposure_adjustment:
            adjustments.append(exposure_adjustment)

        # Check contrast
        contrast_adjustment = self._analyze_contrast(image)
        if contrast_adjustment:
            adjustments.append(contrast_adjustment)

        # Check color balance
        color_adjustment = self._analyze_color_balance(image)
        if color_adjustment:
            adjustments.append(color_adjustment)

        # Check sharpness
        sharpness_adjustment = self._analyze_sharpness(image)
        if sharpness_adjustment:
            adjustments.append(sharpness_adjustment)

        return adjustments

    def _analyze_exposure(self, image: np.ndarray) -> Optional[Adjustment]:
        """Analyze image exposure and recommend adjustment if needed.

        Args:
            image: Input image

        Returns:
            Exposure adjustment or None
        """
        # Convert to grayscale
        if len(image.shape) > 2:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image

        # Calculate histogram
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])

        # Normalize histogram
        hist_norm = hist / (image.shape[0] * image.shape[1])

        # Calculate cumulative distribution function
        cdf = hist_norm.cumsum()

        # Check for underexposure (too many dark pixels)
        if cdf[64] > 0.5:  # More than 50% of pixels are in the darkest quarter
            return Adjustment(
                parameter="exposure",
                suggested=1.0,
                unit="EV",
                description="Image is underexposed, increase brightness"
            )

        # Check for overexposure (too many bright pixels)
        if cdf[192] < 0.5:  # Less than 50% of pixels are below the brightest quarter
            return Adjustment(
                parameter="exposure",
                suggested=-0.5,
                unit="EV",
                description="Image is overexposed, decrease brightness"
            )

        return None

    def _analyze_contrast(self, image: np.ndarray) -> Optional[Adjustment]:
        """Analyze image contrast and recommend adjustment if needed.

        Args:
            image: Input image

        Returns:
            Contrast adjustment or None
        """
        # Convert to grayscale
        if len(image.shape) > 2:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image

        # Calculate standard deviation as a measure of contrast
        std_dev = np.std(gray)

        # Low contrast
        if std_dev < 30:
            return Adjustment(
                parameter="contrast",
                suggested=1.3,
                unit="multiplier",
                description="Image has low contrast, increase for better definition"
            )

        # High contrast
        if std_dev > 80:
            return Adjustment(
                parameter="contrast",
                suggested=0.8,
                unit="multiplier",
                description="Image has high contrast, decrease for better balance"
            )

        return None

    def _analyze_color_balance(self, image: np.ndarray) -> Optional[Adjustment]:
        """Analyze color balance and recommend adjustment if needed.

        Args:
            image: Input image

        Returns:
            Color balance adjustment or None
        """
        # Skip grayscale images
        if len(image.shape) <= 2 or image.shape[2] == 1:
            return None

        # Calculate average color
        avg_color = np.mean(image, axis=(0, 1))

        # Check for color cast
        r, g, b = avg_color

        # Calculate color balance
        max_channel = max(r, g, b)
        min_channel = min(r, g, b)

        # If there's a significant color cast
        if max_channel - min_channel > 15:
            # Blue cast (too cool)
            if b > r and b > g:
                return Adjustment(
                    parameter="temperature",
                    suggested=0.2,
                    unit="shift",
                    description="Image has a cool/blue cast, warm it up"
                )

            # Red cast (too warm)
            if r > b and r > g:
                return Adjustment(
                    parameter="temperature",
                    suggested=-0.2,
                    unit="shift",
                    description="Image has a warm/red cast, cool it down"
                )

        return None

    def _analyze_sharpness(self, image: np.ndarray) -> Optional[Adjustment]:
        """Analyze image sharpness and recommend adjustment if needed.

        Args:
            image: Input image

        Returns:
            Sharpness adjustment or None
        """
        # Convert to grayscale
        if len(image.shape) > 2:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image

        # Calculate Laplacian variance as a measure of sharpness
        laplacian = cv2.Laplacian(gray, cv2.CV_64F)
        sharpness = np.var(laplacian)

        # Low sharpness
        if sharpness < 100:
            return Adjustment(
                parameter="sharpening",
                suggested=0.5,
                unit="strength",
                description="Image is soft, increase sharpness"
            )

        return None

# Implementation of ImageExecutor class
class ImageExecutor:
    """Applies adjustments to images."""

    def __init__(self, config: Optional[Dict[str, Any]] = None):
        """Initialize the image executor.

        Args:
            config: Configuration dictionary
        """
        self.config = config or {}
        self._validate_config()

    def _validate_config(self) -> None:
        """Validate and set default configuration parameters."""
        defaults = {
            'max_adjustment_intensity': 1.0,  # Scale factor for all adjustments (0-1)
        }

        for key, value in defaults.items():
            if key not in self.config:
                self.config[key] = value

    def apply(self, image_source: Union[str, np.ndarray], adjustments: List[Adjustment], style: str = None) -> np.ndarray:
        """Apply adjustments to an image.

        Args:
            image_source: Input image as file path or numpy array
            adjustments: List of adjustments to apply
            style: Optional style preset name

        Returns:
            Processed image as numpy array
        """
        # Load the image if it's a file path
        image = load_image(image_source)
        result = image.copy()

        # First apply the style preset if specified
        if style:
            result = self._apply_style(result, style)

        # Then apply individual adjustments
        for adjustment in adjustments:
            result = self._apply_adjustment(result, adjustment)

        return result

    def _apply_style(self, image: np.ndarray, style: str) -> np.ndarray:
        """Apply a style preset to an image.

        Args:
            image: Input image
            style: Style preset name

        Returns:
            Processed image
        """
        # Copy the image to avoid modifying the original
        result = image.copy()

        # Apply the appropriate style preset
        style_lower = style.lower().replace(' ', '-')

        if style_lower == "auto-enhance":
            result = self._style_auto_enhance(result)
        elif style_lower == "portrait":
            result = self._style_portrait(result)
        elif style_lower == "vintage":
            result = self._style_vintage(result)
        # Add new cinematic styles
        elif style_lower == "cinematic-teal-orange" or style_lower == "cinematic-teal-&-orange":
            result = self._style_cinematic_teal_orange(result)
        elif style_lower == "film-noir":
            result = self._style_film_noir(result)
        elif style_lower == "anamorphic":
            result = self._style_anamorphic(result)
        elif style_lower == "blockbuster":
            result = self._style_blockbuster(result)
        elif style_lower == "dreamy":
            result = self._style_dreamy(result)
        else:
            logger.warning(f"Unknown style preset: {style}")

        return result

    def _style_auto_enhance(self, image: np.ndarray) -> np.ndarray:
        """Apply the Auto-Enhance style."""
        # Convert to LAB color space
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

        # Split channels
        l, a, b = cv2.split(lab)

        # Apply CLAHE to L channel
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        l = clahe.apply(l)

        # Merge channels
        lab = cv2.merge([l, a, b])

        # Convert back to RGB
        enhanced = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

        # Apply slight sharpening
        enhanced = self._apply_sharpening(enhanced, 0.2)

        return enhanced

    def _style_portrait(self, image: np.ndarray) -> np.ndarray:
        """Apply the Portrait style."""
        # Start with auto-enhance
        result = self._style_auto_enhance(image)

        # Apply skin smoothing (bilateral filter)
        # Convert to float32 for better precision
        float_img = result.astype(np.float32) / 255.0
        # Apply bilateral filter for edge-preserving smoothing
        smoothed = cv2.bilateralFilter(float_img, 9, 75, 75)
        # Convert back to uint8
        smoothed = (smoothed * 255).astype(np.uint8)

        # Warm up the image slightly
        result = self._apply_temperature(smoothed, 0.1)

        # Slightly increase saturation
        result = self._apply_saturation(result, 0.1)

        return result

    def _style_vintage(self, image: np.ndarray) -> np.ndarray:
        """Apply the Vintage style."""
        # Create a vintage look
        # Slightly reduce contrast
        result = self._apply_contrast(image, 0.8)

        # Apply a slight sepia tone
        sepia = np.array([
            [0.393, 0.769, 0.189],
            [0.349, 0.686, 0.168],
            [0.272, 0.534, 0.131]
        ])

        # Convert to float for matrix multiplication
        float_img = result.astype(np.float32) / 255.0
        sepia_img = np.zeros_like(float_img)

        # Apply sepia matrix
        for i in range(3):
            sepia_img[:, :, i] = np.sum(float_img * sepia[i], axis=2)

        # Clip values to valid range
        sepia_img = np.clip(sepia_img, 0, 1.0)

        # Convert back to uint8
        result = (sepia_img * 255).astype(np.uint8)

        # Add slight vignette
        height, width = result.shape[:2]

        # Create vignette mask
        x = np.linspace(-1, 1, width)
        y = np.linspace(-1, 1, height)
        x, y = np.meshgrid(x, y)
        radius = np.sqrt(x**2 + y**2)

        # Create vignette
        vignette = np.clip(1.0 - radius * 0.5, 0, 1.0)
        vignette = np.dstack([vignette] * 3)

        # Apply vignette
        result = (result.astype(np.float32) * vignette).astype(np.uint8)

        return result

    def _style_cinematic_teal_orange(self, image: np.ndarray) -> np.ndarray:
        """Apply the Cinematic Teal & Orange style."""
        # Enhance contrast first
        result = self._apply_contrast(image, 1.3)

        # Convert to float for processing
        img_float = result.astype(np.float32) / 255.0

        # Split into channels
        b, g, r = cv2.split(img_float)

        # Manipulate shadows (blue/teal) and highlights (orange)
        # Boost blue in shadows
        shadows = 1.0 - ((r + g) / 2.0)  # Approximate shadow areas
        b_new = b + (shadows * 0.15)  # Add blue to shadows

        # Boost orange in highlights
        highlights = ((r + g) / 2.0)  # Approximate highlight areas
        r_new = r + (highlights * 0.1)  # Add red to highlights
        g_new = g + (highlights * 0.05)  # Add a bit of green for orange

        # Clip to valid range
        b_new = np.clip(b_new, 0, 1.0)
        g_new = np.clip(g_new, 0, 1.0)
        r_new = np.clip(r_new, 0, 1.0)

        # Merge channels
        result = cv2.merge([b_new, g_new, r_new])

        # Convert back to uint8
        result = (result * 255).astype(np.uint8)

        # Apply slight sharpening
        result = self._apply_sharpening(result, 0.25)

        # Add subtle vignette
        height, width = result.shape[:2]
        x = np.linspace(-1, 1, width)
        y = np.linspace(-1, 1, height)
        x, y = np.meshgrid(x, y)
        radius = np.sqrt(x**2 + y**2)
        vignette = np.clip(1.0 - radius * 0.3, 0, 1.0)
        vignette = np.dstack([vignette] * 3)
        result = (result.astype(np.float32) * vignette).astype(np.uint8)

        return result

    def _style_film_noir(self, image: np.ndarray) -> np.ndarray:
        """Apply the Film Noir style."""
        # First convert to black and white with high contrast
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Apply CLAHE for better contrast distribution
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        gray = clahe.apply(gray)

        # Enhance contrast
        gray = cv2.convertScaleAbs(gray, alpha=1.5, beta=-10)

        # Convert back to RGB
        result = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)

        # Add strong vignette for dramatic effect
        height, width = result.shape[:2]
        x = np.linspace(-1, 1, width)
        y = np.linspace(-1, 1, height)
        x, y = np.meshgrid(x, y)
        radius = np.sqrt(x**2 + y**2)
        vignette = np.clip(1.0 - radius * 0.8, 0, 1.0)
        vignette = np.dstack([vignette] * 3)
        result = (result.astype(np.float32) * vignette).astype(np.uint8)

        # Add film grain
        noise = np.random.normal(0, 0.03, result.shape).astype(np.float32)
        result = np.clip(result.astype(np.float32) / 255.0 + noise, 0, 1) * 255
        result = result.astype(np.uint8)

        return result

    def _style_anamorphic(self, image: np.ndarray) -> np.ndarray:
        """Apply the Anamorphic style."""
        # Enhance contrast and color
        result = self._apply_contrast(image, 1.25)
        result = self._apply_saturation(result, 0.1)

        # Shift color balance slightly toward blue
        result = self._apply_temperature(result, -0.1)

        # Add horizontal lens flare effect (blue streak)
        height, width = result.shape[:2]

        # Create horizontal flare in random position
        flare_y = np.random.randint(height // 4, 3 * height // 4)
        flare = np.zeros_like(result, dtype=np.float32)

        # Create horizontal blue streak
        for y in range(max(0, flare_y - 5), min(height, flare_y + 6)):
            intensity = 1.0 - abs(y - flare_y) / 5.0
            for x in range(width):
                # Blue-tinted flare with falloff from center
                dist_from_center = abs(x - width // 2) / (width // 2)
                flare_intensity = intensity * (1.0 - dist_from_center**2) * 0.4
                flare[y, x, 0] = flare_intensity * 0.8  # Blue channel
                flare[y, x, 1] = flare_intensity * 0.3  # Green channel
                flare[y, x, 2] = flare_intensity * 0.2  # Red channel

        # Add flare to image
        result = np.clip(result.astype(np.float32) + flare * 255, 0, 255).astype(np.uint8)

        # Add letterbox effect to simulate widescreen
        letterbox_height = height // 6
        result[0:letterbox_height, :] = [0, 0, 0]
        result[height - letterbox_height:height, :] = [0, 0, 0]

        return result

    def _style_blockbuster(self, image: np.ndarray) -> np.ndarray:
        """Apply the Blockbuster style."""
        # Increase exposure slightly
        result = self._apply_exposure(image, 0.1)

        # Enhance contrast dramatically
        result = self._apply_contrast(result, 1.4)

        # Boost saturation for vivid colors
        result = self._apply_saturation(result, 0.25)

        # Enhance sharpness
        result = self._apply_sharpening(result, 0.3)

        # Add color tint based on dominant colors (typical blockbuster color grading)
        # Convert to HSV for easier color manipulation
        hsv = cv2.cvtColor(result, cv2.COLOR_RGB2HSV)
        h, s, v = cv2.split(hsv)

        # Shift colors slightly toward blue/cyan for shadows and orange/yellow for highlights
        # This creates the modern blockbuster look
        mask = v < 128  # Shadow areas
        h[mask] = np.clip(h[mask] + 10, 0, 179)  # Shift toward blue-cyan

        mask = v >= 128  # Highlight areas
        h[mask] = np.clip(h[mask] - 10, 0, 179)  # Shift toward orange-yellow

        # Merge channels
        hsv = cv2.merge([h, s, v])

        # Convert back to RGB
        result = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

        # Add subtle vignette
        height, width = result.shape[:2]
        x = np.linspace(-1, 1, width)
        y = np.linspace(-1, 1, height)
        x, y = np.meshgrid(x, y)
        radius = np.sqrt(x**2 + y**2)
        vignette = np.clip(1.0 - radius * 0.2, 0, 1.0)
        vignette = np.dstack([vignette] * 3)
        result = (result.astype(np.float32) * vignette).astype(np.uint8)

        return result

    def _style_dreamy(self, image: np.ndarray) -> np.ndarray:
        """Apply the Dreamy style."""
        # Reduce contrast
        result = self._apply_contrast(image, 0.85)

        # Add warmth
        result = self._apply_temperature(result, 0.15)

        # Apply soft glow effect
        # Create blurred version
        blur = cv2.GaussianBlur(result, (21, 21), 0)

        # Blend with original (soft glow effect)
        result = cv2.addWeighted(result, 0.7, blur, 0.3, 0)

        # Apply noise reduction
        result = self._apply_noise_reduction(result, 0.4)

        # Add dreamy haze/fog effect
        height, width = result.shape[:2]
        fog = np.ones_like(result) * 255  # White fog

        # Create fog gradient
        for y in range(height):
            for x in range(width):
                # Calculate distance from top left
                dist = np.sqrt((x / width)**2 + (y / height)**2)
                fog_alpha = 0.15 * (1.0 - dist)  # Stronger fog in top-left corner
                result[y, x] = cv2.addWeighted(result[y, x], 1 - fog_alpha, fog[y, x], fog_alpha, 0)

        # Add vignette for dreamy border effect
        x = np.linspace(-1, 1, width)
        y = np.linspace(-1, 1, height)
        x, y = np.meshgrid(x, y)
        radius = np.sqrt(x**2 + y**2)
        vignette = np.clip(1.0 - radius * 0.4, 0, 1.0)
        vignette = np.dstack([vignette] * 3)
        result = (result.astype(np.float32) * vignette).astype(np.uint8)

        return result

    def _apply_adjustment(self, image: np.ndarray, adjustment: Adjustment) -> np.ndarray:
        """Apply a single adjustment to an image.

        Args:
            image: Input image
            adjustment: Adjustment to apply

        Returns:
            Processed image
        """
        # Scale the adjustment by the max intensity
        intensity = self.config['max_adjustment_intensity']

        # Apply the appropriate adjustment based on parameter
        if adjustment.parameter == "exposure":
            return self._apply_exposure(image, adjustment.suggested * intensity)
        elif adjustment.parameter == "contrast":
            return self._apply_contrast(image, adjustment.suggested)
        elif adjustment.parameter == "noise_reduction":
            return self._apply_noise_reduction(image, adjustment.suggested * intensity)
        elif adjustment.parameter == "sharpening":
            return self._apply_sharpening(image, adjustment.suggested * intensity)
        elif adjustment.parameter == "saturation":
            return self._apply_saturation(image, adjustment.suggested * intensity)
        elif adjustment.parameter == "temperature":
            return self._apply_temperature(image, adjustment.suggested * intensity)
        else:
            logger.warning(f"Unknown adjustment parameter: {adjustment.parameter}")
            return image

    def _apply_exposure(self, image: np.ndarray, ev: float) -> np.ndarray:
        """Apply exposure adjustment.

        Args:
            image: Input image
            ev: Exposure value adjustment in EV

        Returns:
            Processed image
        """
        # Convert to float
        float_img = image.astype(np.float32) / 255.0

        # Calculate multiplier from EV
        multiplier = 2 ** ev

        # Apply adjustment
        result = float_img * multiplier

        # Clip values
        result = np.clip(result, 0, 1.0)

        # Convert back to uint8
        return (result * 255).astype(np.uint8)

    def _apply_contrast(self, image: np.ndarray, multiplier: float) -> np.ndarray:
        """Apply contrast adjustment.

        Args:
            image: Input image
            multiplier: Contrast multiplier

        Returns:
            Processed image
        """
        # Convert to float
        float_img = image.astype(np.float32) / 255.0

        # Calculate mean
        mean = np.mean(float_img, axis=(0, 1), keepdims=True)

        # Apply contrast adjustment (center around mean)
        result = (float_img - mean) * multiplier + mean

        # Clip values
        result = np.clip(result, 0, 1.0)

        # Convert back to uint8
        return (result * 255).astype(np.uint8)

    def _apply_noise_reduction(self, image: np.ndarray, strength: float) -> np.ndarray:
        """Apply noise reduction.

        Args:
            image: Input image
            strength: Strength of noise reduction (0-1)

        Returns:
            Processed image
        """
        # Apply bilateral filter
        # Scale strength to appropriate range for bilateral filter
        d = int(3 + strength * 7)  # Diameter of filter, 3-10
        sigma_color = 10 + strength * 90  # 10-100
        sigma_space = 10 + strength * 90  # 10-100

        return cv2.bilateralFilter(image, d, sigma_color, sigma_space)

    def _apply_sharpening(self, image: np.ndarray, strength: float) -> np.ndarray:
        """Apply sharpening.

        Args:
            image: Input image
            strength: Strength of sharpening (0-1)

        Returns:
            Processed image
        """
        # Apply unsharp mask
        blurred = cv2.GaussianBlur(image, (0, 0), 3)
        sharpened = cv2.addWeighted(
            image, 1.0 + strength, 
            blurred, -strength, 
            0
        )

        return sharpened

    def _apply_saturation(self, image: np.ndarray, adjustment: float) -> np.ndarray:
        """Apply saturation adjustment.

        Args:
            image: Input image
            adjustment: Saturation adjustment (-1 to 1)

        Returns:
            Processed image
        """
        # Convert to HSV
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV).astype(np.float32)

        # Calculate multiplier (1.0 means no change)
        if adjustment > 0:
            # Increase saturation
            multiplier = 1.0 + adjustment
        else:
            # Decrease saturation
            multiplier = 1.0 + adjustment

        # Apply multiplier to saturation channel
        hsv[:, :, 1] = np.clip(hsv[:, :, 1] * multiplier, 0, 255)

        # Convert back to BGR
        return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)

    def _apply_temperature(self, image: np.ndarray, adjustment: float) -> np.ndarray:
        """Apply color temperature adjustment.

        Args:
            image: Input image
            adjustment: Temperature adjustment (-1 to 1)

        Returns:
            Processed image
        """
        # Copy the image
        result = image.copy().astype(np.float32)

        # Apply temperature adjustment
        if adjustment > 0:
            # Warm up (increase red, decrease blue)
            result[:, :, 0] = np.clip(result[:, :, 0] * (1 + adjustment * 0.2), 0, 255)  # Red
            result[:, :, 2] = np.clip(result[:, :, 2] * (1 - adjustment * 0.1), 0, 255)  # Blue
        else:
            # Cool down (decrease red, increase blue)
            adjustment = abs(adjustment)
            result[:, :, 0] = np.clip(result[:, :, 0] * (1 - adjustment * 0.1), 0, 255)  # Red
            result[:, :, 2] = np.clip(result[:, :, 2] * (1 + adjustment * 0.2), 0, 255)  # Blue

        return result.astype(np.uint8)

# Implementation of analyze_image and apply_adjustments functions
def analyze_image(image_source):
    """Analyze an image and return recommended adjustments.

    Args:
        image_source: Path to the image or a numpy array

    Returns:
        List of recommended adjustments
    """
    analyzer = ImageAnalyzer()
    return analyzer.analyze(image_source)

def apply_adjustments(image_source, adjustments, style=None):
    """Apply adjustments and/or a style to an image.

    Args:
        image_source: Path to the image or a numpy array
        adjustments: List of adjustments to apply
        style: Optional style preset name

    Returns:
        Processed image as a numpy array
    """
    executor = ImageExecutor()
    return executor.apply(image_source, adjustments, style)

# Implementation of AIImageAnalyzer class
class AIImageAnalyzer:
    """Uses AI models to analyze image content and make intelligent suggestions."""

    def __init__(self, config: Optional[Dict[str, Any]] = None):
        """Initialize the AI image analyzer.

        Args:
            config: Configuration dictionary for model paths and parameters
        """
        self.config = config or {}
        self._validate_config()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Initialize models lazily to reduce startup time
        self.scene_model = None
        self.face_model = None
        self.object_model = None

    def _validate_config(self) -> None:
        """Validate and set default configuration parameters."""
        defaults = {
            'model_path': 'default_model',  # Use a default model in contained environment
            'detection_threshold': 0.5,
            'max_objects': 10,
        }

        for key, value in defaults.items():
            if key not in self.config:
                self.config[key] = value

    def _load_scene_model(self):
        """Load the scene classification model."""
        try:
            # In a real implementation, we would use a model like ResNet pre-trained on Places365
            # For demo purposes, we'll simulate loading a model
            logger.info("Loading scene classification model")
            self.scene_model = True  # Simulate successful loading
        except Exception as e:
            logger.error(f"Failed to load scene model: {e}")
            self.scene_model = None

    def _load_face_model(self):
        """Load the face detection model."""
        try:
            # In a real implementation, load a face detection model like MTCNN
            # For now, we'll use OpenCV's built-in face detector
            logger.info("Loading face detection model")
            self.face_model = cv2.CascadeClassifier(
                cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
            )
        except Exception as e:
            logger.error(f"Failed to load face model: {e}")
            self.face_model = None

    def _load_object_model(self):
        """Load the object detection model."""
        try:
            # In a real implementation, use a model like YOLO or Faster R-CNN
            # For now, we'll simulate loading a model
            logger.info("Loading object detection model")
            self.object_model = True  # Simulate successful loading
        except Exception as e:
            logger.error(f"Failed to load object model: {e}")
            self.object_model = None

    def analyze(self, image: np.ndarray) -> Tuple[List[Adjustment], Dict[str, Any]]:
        """Analyze image content and suggest appropriate adjustments.

        Args:
            image: Input image as numpy array

        Returns:
            A tuple containing (adjustments list, analysis metadata)
        """
        # Ensure models are loaded
        if self.scene_model is None:
            self._load_scene_model()
        if self.face_model is None:
            self._load_face_model()
        if self.object_model is None:
            self._load_object_model()

        # Initialize analysis results
        analysis = {
            'scene_type': None,
            'has_faces': False,
            'face_count': 0,
            'objects': [],
            'lighting_condition': None,
            'color_palette': None,
        }

        # Analyze scene type
        scene_type = self._analyze_scene(image)
        analysis['scene_type'] = scene_type

        # Detect faces
        faces = self._detect_faces(image)
        analysis['has_faces'] = len(faces) > 0
        analysis['face_count'] = len(faces)

        # Detect objects
        objects = self._detect_objects(image)
        analysis['objects'] = objects

        # Analyze lighting conditions
        lighting = self._analyze_lighting(image)
        analysis['lighting_condition'] = lighting

        # Extract color palette
        color_palette = self._extract_color_palette(image)
        analysis['color_palette'] = color_palette

        # Generate intelligent adjustment recommendations
        adjustments = self._generate_adjustments(image, analysis)

        return adjustments, analysis

    def _analyze_scene(self, image: np.ndarray) -> str:
        """Classify the scene type in the image.

        Args:
            image: Input image

        Returns:
            Scene type as string
        """
        # In a real implementation, use the scene model to classify the image
        # For now, use simple heuristics for demonstration

        # Convert to RGB if not already
        if len(image.shape) == 2 or image.shape[2] == 1:
            # Grayscale image
            return "unknown"

        # Simple color-based heuristics
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        h, s, v = cv2.split(hsv)

        # Check for landscapes based on color distribution
        blue_sky = np.mean(h[v > 200]) > 100 and np.mean(h[v > 200]) < 140
        green_dominant = np.mean(h) > 35 and np.mean(h) < 85 and np.mean(s) > 50

        if blue_sky and green_dominant:
            return "landscape"
        elif blue_sky:
            return "sky"
        elif green_dominant:
            return "nature"

        # Check for indoor/urban scenes
        if np.mean(v) < 100:
            return "indoor"

        # Default fallback
        return "general"

    def _detect_faces(self, image: np.ndarray) -> List[Dict[str, Any]]:
        """Detect faces in the image.

        Args:
            image: Input image

        Returns:
            List of face detection results
        """
        if self.face_model is None:
            return []

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Detect faces
        faces = self.face_model.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )

        # Format results
        face_results = []
        for (x, y, w, h) in faces:
            face_results.append({
                'bbox': (x, y, x+w, y+h),
                'confidence': 0.9,  # Placeholder confidence
            })

        return face_results

    def _detect_objects(self, image: np.ndarray) -> List[Dict[str, Any]]:
        """Detect objects in the image.

        Args:
            image: Input image

        Returns:
            List of object detection results
        """
        # For demonstration, return simulated objects based on scene type
        scene_type = self._analyze_scene(image)

        if scene_type == "landscape":
            return [
                {'class': 'mountain', 'confidence': 0.8},
                {'class': 'tree', 'confidence': 0.7},
                {'class': 'sky', 'confidence': 0.9}
            ]
        elif scene_type == "indoor":
            return [
                {'class': 'chair', 'confidence': 0.6},
                {'class': 'table', 'confidence': 0.5}
            ]

        # Default fallback
        return []

    def _analyze_lighting(self, image: np.ndarray) -> str:
        """Analyze lighting conditions in the image.

        Args:
            image: Input image

        Returns:
            Lighting condition as string
        """
        # Convert to grayscale
        if len(image.shape) > 2:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image

        # Calculate histogram
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])

        # Analyze histogram for lighting conditions
        dark_pixels = np.sum(hist[:64])
        mid_pixels = np.sum(hist[64:192])
        bright_pixels = np.sum(hist[192:])

        total_pixels = dark_pixels + mid_pixels + bright_pixels

        dark_ratio = dark_pixels / total_pixels
        bright_ratio = bright_pixels / total_pixels

        if dark_ratio > 0.5:
            return "low_light"
        elif bright_ratio > 0.5:
            return "bright"
        else:
            return "normal"

    def _extract_color_palette(self, image: np.ndarray) -> List[List[int]]:
        """Extract dominant color palette from the image.

        Args:
            image: Input image

        Returns:
            List of RGB color values
        """
        # Resize image for faster processing
        small = cv2.resize(image, (100, 100))

        # Reshape for k-means
        pixels = small.reshape(-1, 3)
        pixels = np.float32(pixels)

        # Define criteria and apply k-means
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, 0.1)
        k = 5  # Number of colors to extract
        _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

        # Convert centers to integer RGB values
        centers = np.uint8(centers)

        # Count pixels in each cluster
        counts = np.bincount(labels.flatten())

        # Sort colors by frequency
        sorted_indices = np.argsort(counts)[::-1]
        palette = [centers[i].tolist() for i in sorted_indices]

        return palette

    def _generate_adjustments(self, image: np.ndarray, analysis: Dict[str, Any]) -> List[Adjustment]:
        """Generate adjustment recommendations based on image analysis.

        Args:
            image: Input image
            analysis: Analysis results

        Returns:
            List of recommended adjustments
        """
        adjustments = []

        # Generate scene-specific adjustments
        scene_type = analysis.get('scene_type', 'general')
        if scene_type == "landscape":
            # Enhance blues for sky and greens for vegetation
            adjustments.append(Adjustment(
                parameter="saturation",
                suggested=0.2,
                unit="increase",
                description="Enhance landscape colors"
            ))
            adjustments.append(Adjustment(
                parameter="contrast",
                suggested=1.15,
                unit="multiplier",
                description="Boost landscape contrast"
            ))
        elif scene_type == "indoor":
            # Indoor scenes often need white balance correction
            adjustments.append(Adjustment(
                parameter="temperature",
                suggested=0.1,
                unit="shift",
                description="Correct indoor lighting"
            ))

        # Lighting-specific adjustments
        lighting = analysis.get('lighting_condition', 'normal')
        if lighting == "low_light":
            # Brighten dark images
            adjustments.append(Adjustment(
                parameter="exposure",
                suggested=0.5,
                unit="EV",
                description="Brighten dark image"
            ))
            # Reduce noise in low light
            adjustments.append(Adjustment(
                parameter="noise_reduction",
                suggested=0.4,
                unit="strength",
                description="Reduce low-light noise"
            ))
        elif lighting == "bright":
            # Recover highlights in bright images
            adjustments.append(Adjustment(
                parameter="exposure",
                suggested=-0.2,
                unit="EV",
                description="Recover bright highlights"
            ))

        # Portrait-specific adjustments
        if analysis.get('has_faces', False):
            # Enhance portraits
            adjustments.append(Adjustment(
                parameter="temperature",
                suggested=0.1,
                unit="shift",
                description="Warm skin tones"
            ))
            # Subtle skin smoothing
            adjustments.append(Adjustment(
                parameter="noise_reduction",
                suggested=0.3,
                unit="strength",
                description="Smooth skin details"
            ))

        return adjustments

    def suggest_style(self, image: np.ndarray, analysis: Optional[Dict[str, Any]] = None) -> str:
        """Suggest an appropriate style preset based on image content.

        Args:
            image: Input image
            analysis: Optional pre-computed analysis results

        Returns:
            Name of the recommended style preset
        """
        # Run analysis if not provided
        if analysis is None:
            _, analysis = self.analyze(image)

        # Suggest style based on content
        scene_type = analysis.get('scene_type', 'general')
        has_faces = analysis.get('has_faces', False)
        lighting = analysis.get('lighting_condition', 'normal')

        # Style selection logic
        if has_faces:
            if lighting == "low_light":
                return "Film Noir"  # Dramatic portrait style
            else:
                return "Portrait"  # Standard portrait style
        elif scene_type in ["landscape", "nature"]:
            return "Cinematic Teal & Orange"  # Good for landscapes
        elif scene_type == "indoor" and lighting == "low_light":
            return "Anamorphic"  # Good for indoor/low light
        elif lighting == "bright":
            return "Blockbuster"  # Vivid style for bright scenes

        # Default fallback
        return "Auto-Enhance"

# Implementation of NLProcessor class
class NLProcessor:
    """Processes natural language instructions for photo editing."""

    def __init__(self, config: Optional[Dict[str, Any]] = None):
        """Initialize the natural language processor.

        Args:
            config: Configuration dictionary
        """
        self.config = config or {}
        self._validate_config()

        # Function registry maps function names to actual functions
        self.function_registry = {}

    def _validate_config(self) -> None:
        """Validate and set default configuration parameters."""
        defaults = {
            'api_key': os.environ.get('OPENAI_API_KEY', ''),
            'model': 'gpt-4',
            'max_tokens': 150,
        }

        for key, value in defaults.items():
            if key not in self.config:
                self.config[key] = value

    def register_function(self, name: str, func: Callable, description: str, parameters: Dict[str, Any]):
        """Register a function that can be called from natural language.

        Args:
            name: Function name
            func: Function to call
            description: Description of what the function does
            parameters: Parameter schema for the function
        """
        self.function_registry[name] = {
            'function': func,
            'description': description,
            'parameters': parameters
        }

    def process(self, image: np.ndarray, instruction: str) -> Tuple[np.ndarray, Dict[str, Any]]:
        """Process a natural language instruction and apply it to an image.

        Args:
            image: Input image
            instruction: Natural language instruction

        Returns:
            Tuple of (processed image, metadata)
        """
        # Extract operations from instruction
        operations = self._parse_instruction(instruction)

        # Initialize metadata
        metadata = {
            'instruction': instruction,
            'functions_called': [],
            'errors': []
        }

        # Apply each operation in sequence
        result = image.copy()
        for op in operations:
            try:
                # Log the function call
                metadata['functions_called'].append({
                    'name': op['name'],
                    'args': op['arguments']
                })

                # Call the function
                if op['name'] in self.function_registry:
                    func_info = self.function_registry[op['name']]
                    func = func_info['function']
                    result = func(result, **op['arguments'])
                else:
                    metadata['errors'].append(f"Unknown function: {op['name']}")

            except Exception as e:
                metadata['errors'].append(f"Error in {op['name']}: {str(e)}")
                logger.error(f"Error applying operation {op['name']}: {e}")

        return result, metadata

    def _parse_instruction(self, instruction: str) -> List[Dict[str, Any]]:
        """Parse natural language instruction into specific operations.

        Args:
            instruction: Natural language instruction

        Returns:
            List of operations (function name and arguments)
        """
        try:
            # In a real implementation, this would call an LLM API with function calling
            # For demo purposes, we'll simulate the function calling behavior

            # Generate JSON schema for all registered functions
            tools = []
            for name, info in self.function_registry.items():
                tools.append({
                    "type": "function",
                    "function": {
                        "name": name,
                        "description": info['description'],
                        "parameters": info['parameters']
                    }
                })

            # For demo purposes, let's simulate an LLM response based on the instruction
            return self._simulate_function_calls(instruction)

        except Exception as e:
            logger.error(f"Error parsing instruction: {e}")
            return []

    def _simulate_function_calls(self, instruction: str) -> List[Dict[str, Any]]:
        """Simulate function calls based on the instruction (demo only).

        Args:
            instruction: Natural language instruction

        Returns:
            List of simulated function calls
        """
        instruction_lower = instruction.lower()
        operations = []

        # Handle brightness/exposure adjustments
        if any(term in instruction_lower for term in ['bright', 'exposure', 'darker', 'lighter']):
            amount = 0.3 if 'bright' in instruction_lower or 'lighter' in instruction_lower else -0.3

            # Adjust magnitude based on modifiers
            if 'slightly' in instruction_lower or 'subtle' in instruction_lower:
                amount *= 0.5
            elif 'very' in instruction_lower or 'much' in instruction_lower:
                amount *= 1.5

            operations.append({
                'name': 'adjust_exposure',
                'arguments': {'amount': amount}
            })

        # Handle contrast adjustments
        if 'contrast' in instruction_lower:
            increase = 'increase' in instruction_lower or 'more' in instruction_lower
            amount = 1.2 if increase else 0.8

            # Adjust magnitude based on modifiers
            if 'slightly' in instruction_lower or 'subtle' in instruction_lower:
                amount = 1.1 if increase else 0.9
            elif 'very' in instruction_lower or 'much' in instruction_lower or 'dramatic' in instruction_lower:
                amount = 1.4 if increase else 0.7

            operations.append({
                'name': 'adjust_contrast',
                'arguments': {'multiplier': amount}
            })

        # Handle saturation/vibrance adjustments
        if any(term in instruction_lower for term in ['saturation', 'vibrance', 'vibrant', 'colorful']):
            increase = not ('reduce' in instruction_lower or 'less' in instruction_lower)
            amount = 0.2 if increase else -0.2

            # Adjust magnitude based on modifiers
            if 'slightly' in instruction_lower or 'subtle' in instruction_lower:
                amount *= 0.5
            elif 'very' in instruction_lower or 'much' in instruction_lower:
                amount *= 1.5

            operations.append({
                'name': 'adjust_saturation',
                'arguments': {'adjustment': amount}
            })

        # Handle temperature/warmth adjustments
        if any(term in instruction_lower for term in ['warm', 'temperature', 'cool', 'cold']):
            warm = 'warm' in instruction_lower
            amount = 0.15 if warm else -0.15

            # Adjust magnitude based on modifiers
            if 'slightly' in instruction_lower or 'subtle' in instruction_lower:
                amount *= 0.5
            elif 'very' in instruction_lower or 'much' in instruction_lower:
                amount *= 1.5

            operations.append({
                'name': 'adjust_temperature',
                'arguments': {'adjustment': amount}
            })

        # Handle sharpness adjustments
        if any(term in instruction_lower for term in ['sharp', 'clarity', 'detail']):
            increase = not ('reduce' in instruction_lower or 'less' in instruction_lower)
            amount = 0.3 if increase else -0.1

            # Adjust magnitude based on modifiers
            if 'slightly' in instruction_lower or 'subtle' in instruction_lower:
                amount *= 0.7
            elif 'very' in instruction_lower or 'much' in instruction_lower:
                amount *= 1.5

            operations.append({
                'name': 'adjust_sharpness',
                'arguments': {'strength': max(0, amount)}
            })

        # Handle noise reduction
        if 'noise' in instruction_lower or 'grain' in instruction_lower:
            reduce = 'reduce' in instruction_lower or 'less' in instruction_lower or 'remove' in instruction_lower
            amount = 0.4 if reduce else 0.1

            # Adjust magnitude based on modifiers
            if 'slightly' in instruction_lower or 'subtle' in instruction_lower:
                amount *= 0.7
            elif 'very' in instruction_lower or 'much' in instruction_lower:
                amount *= 1.5

            operations.append({
                'name': 'reduce_noise',
                'arguments': {'strength': amount}
            })

        # Handle style-based instructions
        if 'cinematic' in instruction_lower:
            if 'dramatic' in instruction_lower or 'dark' in instruction_lower:
                operations.append({
                    'name': 'apply_style',
                    'arguments': {'style_name': 'Film Noir'}
                })
            elif 'anamorphic' in instruction_lower or 'widescreen' in instruction_lower:
                operations.append({
                    'name': 'apply_style',
                    'arguments': {'style_name': 'Anamorphic'}
                })
            else:
                operations.append({
                    'name': 'apply_style',
                    'arguments': {'style_name': 'Cinematic Teal & Orange'}
                })
        elif 'vintage' in instruction_lower or 'retro' in instruction_lower:
            operations.append({
                'name': 'apply_style',
                'arguments': {'style_name': 'Vintage'}
            })
        elif 'portrait' in instruction_lower:
            operations.append({
                'name': 'apply_style',
                'arguments': {'style_name': 'Portrait'}
            })
        elif 'dreamy' in instruction_lower or 'soft' in instruction_lower:
            operations.append({
                'name': 'apply_style',
                'arguments': {'style_name': 'Dreamy'}
            })
        elif 'dramatic' in instruction_lower or 'action' in instruction_lower:
            operations.append({
                'name': 'apply_style',
                'arguments': {'style_name': 'Blockbuster'}
            })

        # If no specific operations were identified, apply auto-enhance
        if not operations:
            operations.append({
                'name': 'apply_style',
                'arguments': {'style_name': 'Auto-Enhance'}
            })

        return operations

# Implementation of RAGStyleEngine class
class RAGStyleEngine:
    """Recommends and applies styles using RAG techniques."""

    def __init__(self, config: Optional[Dict[str, Any]] = None):
        """Initialize the RAG style engine.

        Args:
            config: Configuration dictionary
        """
        self.config = config or {}
        self._validate_config()

        # Initialize image analyzer
        self.image_analyzer = AIImageAnalyzer()

        # Initialize knowledge base
        self.knowledge_base = self._init_knowledge_base()

        # Initialize embedding database
        self.embedding_db = None
        self._init_embedding_database()

    def _validate_config(self) -> None:
        """Validate and set default configuration parameters."""
        defaults = {
            'knowledge_base_path': os.path.join(os.path.dirname(__file__), 'data', 'style_knowledge.json'),
            'embedding_db_path': os.path.join(os.path.dirname(__file__), 'data', 'style_embeddings.npz'),
            'custom_styles_path': os.path.join(os.path.dirname(__file__), 'data', 'custom_styles'),
        }

        for key, value in defaults.items():
            if key not in self.config:
                self.config[key] = value

        # Create directories if they don't exist
        os.makedirs(os.path.dirname(self.config['knowledge_base_path']), exist_ok=True)
        os.makedirs(os.path.dirname(self.config['embedding_db_path']), exist_ok=True)
        os.makedirs(self.config['custom_styles_path'], exist_ok=True)

    def _init_knowledge_base(self) -> List[Dict[str, Any]]:
        """Initialize the style knowledge base.

        Returns:
            List of style knowledge entries
        """
        # Default knowledge base with cinematography styles and techniques
        default_knowledge = [
            {
                "style_name": "Cinematic Teal & Orange",
                "description": "Classic Hollywood color grading with teal shadows and orange highlights",
                "keywords": ["blockbuster", "cinematic", "movie", "film", "hollywood", "complementary"],
                "examples": ["Transformers", "Marvel movies", "Michael Bay", "action films"],
                "techniques": ["Shadow/highlight color split", "Blue shadows, orange highlights", "High contrast"]
            },
            {
                "style_name": "Film Noir",
                "description": "High contrast black and white with dramatic shadows and film grain",
                "keywords": ["noir", "detective", "dark", "mysterious", "contrast", "dramatic", "moody"],
                "examples": ["The Maltese Falcon", "Citizen Kane", "Double Indemnity"],
                "techniques": ["Hard shadows", "Low-key lighting", "Strong contrast", "Moody atmosphere"]
            },
            {
                "style_name": "Anamorphic",
                "description": "Widescreen cinematic look with lens flares and letterboxing",
                "keywords": ["widescreen", "anamorphic", "cinematic", "lens flare", "letterbox"],
                "examples": ["JJ Abrams films", "Star Trek", "Star Wars", "Sci-fi films"],
                "techniques": ["Horizontal lens flares", "Wide aspect ratio", "Anamorphic lens distortion"]
            },
            {
                "style_name": "Blockbuster",
                "description": "Vibrant colors and high contrast for modern action films",
                "keywords": ["action", "vibrant", "punchy", "bright", "dramatic", "dynamic"],
                "examples": ["Fast & Furious", "Mission Impossible", "Modern action films"],
                "techniques": ["Increased saturation", "High contrast", "Sharper details", "Vibrant colors"]
            },
            {
                "style_name": "Dreamy",
                "description": "Soft, ethereal look with warm tones and gentle glow",
                "keywords": ["soft", "ethereal", "dreamy", "romantic", "fantasy", "peaceful"],
                "examples": ["Romance films", "Fantasy sequences", "Music videos"],
                "techniques": ["Soft focus", "Glow effect", "Reduced contrast", "Warm tones"]
            },
            {
                "style_name": "Vintage",
                "description": "Classic film-inspired look with faded colors and subtle vignette",
                "keywords": ["retro", "vintage", "old", "classic", "film", "nostalgic"],
                "examples": ["Old photographs", "Analog film", "Instagram filters"],
                "techniques": ["Sepia tones", "Reduced contrast", "Faded blacks", "Vignette"]
            },
            {
                "style_name": "Portrait",
                "description": "Optimized for portrait photography with skin tone enhancement",
                "keywords": ["portrait", "person", "face", "skin", "beauty", "professional"],
                "examples": ["Professional portraits", "Headshots", "Fashion photography"],
                "techniques": ["Skin smoothing", "Warm tones", "Subtle contrast", "Detail preservation"]
            }
        ]

        # Try to load existing knowledge base
        try:
            if os.path.exists(self.config['knowledge_base_path']):
                with open(self.config['knowledge_base_path'], 'r') as f:
                    knowledge = json.load(f)
                    logger.info(f"Loaded {len(knowledge)} style knowledge entries")
                    return knowledge
        except Exception as e:
            logger.error(f"Error loading knowledge base: {e}")

        # If loading fails, use default and save it
        try:
            os.makedirs(os.path.dirname(self.config['knowledge_base_path']), exist_ok=True)
            with open(self.config['knowledge_base_path'], 'w') as f:
                json.dump(default_knowledge, f, indent=2)
                logger.info(f"Created default knowledge base with {len(default_knowledge)} entries")
        except Exception as e:
            logger.error(f"Error saving knowledge base: {e}")

        return default_knowledge

    def _init_embedding_database(self) -> None:
        """Initialize the embedding database.

        In a real implementation, this would create embeddings for all style knowledge.
        For the MVP, we'll simulate embeddings.
        """
        # Attempt to load existing embeddings
        try:
            if os.path.exists(self.config['embedding_db_path']):
                # In a real implementation, load saved embeddings
                logger.info("Loaded style embeddings")
                self.embedding_db = True  # Placeholder, simulating successful loading
                return
        except Exception as e:
            logger.error(f"Error loading embeddings: {e}")

        # If loading fails, generate simulated embeddings
        try:
            # In a real implementation, generate embeddings for all entries
            logger.info("Generating simulated style embeddings")
            self.embedding_db = True  # Placeholder, simulating successful generation
        except Exception as e:
            logger.error(f"Error generating embeddings: {e}")

    def recommend_style(self, image: np.ndarray, description: Optional[str] = None) -> List[Dict[str, Any]]:
        """Recommend styles based on image content and optional description.

        Args:
            image: Input image
            description: Optional user description of desired style

        Returns:
            List of recommended styles with reasoning
        """
        # Analyze image content
        _, analysis = self.image_analyzer.analyze(image)

        # Extract key features from analysis
        scene_type = analysis.get('scene_type', 'unknown')
        has_faces = analysis.get('has_faces', False)
        lighting = analysis.get('lighting_condition', 'normal')

        # Match against style knowledge
        if description:
            # For a real implementation, we would use embeddings to find similar styles
            # For now, we'll use keyword matching
            return self._match_by_description(description, scene_type, has_faces, lighting)
        else:
            # Content-based recommendation
            return self._match_by_content(scene_type, has_faces, lighting)

    def _match_by_description(self, description: str, scene_type: str, has_faces: bool, lighting: str) -> List[Dict[str, Any]]:
        """Match styles based on user description and image content.

        Args:
            description: User description of desired style
            scene_type: Detected scene type
            has_faces: Whether faces were detected
            lighting: Detected lighting condition

        Returns:
            List of recommended styles with reasoning
        """
        description_lower = description.lower()
        matches = []

        # Score each style entry
        for entry in self.knowledge_base:
            score = 0
            reasoning = []

            # Match based on style name
            if entry['style_name'].lower() in description_lower:
                score += 10
                reasoning.append(f"Explicitly mentioned {entry['style_name']}")

            # Match based on keywords
            for keyword in entry.get('keywords', []):
                if keyword.lower() in description_lower:
                    score += 3
                    reasoning.append(f"Mentioned '{keyword}'")

            # Match based on examples
            for example in entry.get('examples', []):
                if example.lower() in description_lower:
                    score += 5
                    reasoning.append(f"Referenced '{example}'")

            # Match based on techniques
            for technique in entry.get('techniques', []):
                tech_words = set(technique.lower().split())
                if any(word in description_lower for word in tech_words):
                    score += 2
                    reasoning.append(f"Described technique similar to '{technique}'")

            # Content-based boosting
            if scene_type == "landscape" and entry['style_name'] in ["Cinematic Teal & Orange", "Anamorphic"]:
                score += 2
                reasoning.append("Good match for landscape content")

            if has_faces and entry['style_name'] in ["Portrait", "Film Noir"]:
                score += 2
                reasoning.append("Good match for portrait content")

            if lighting == "low_light" and entry['style_name'] in ["Film Noir", "Anamorphic"]:
                score += 2
                reasoning.append("Good match for low-light conditions")

            # Add to matches if above threshold
            if score > 0:
                matches.append({
                    'style': entry['style_name'],
                    'description': entry['description'],
                    'score': score,
                    'reasoning': reasoning
                })

        # Sort by score and return top matches
        matches.sort(key=lambda x: x['score'], reverse=True)
        return matches[:3]  # Return top 3 matches

    def _match_by_content(self, scene_type: str, has_faces: bool, lighting: str) -> List[Dict[str, Any]]:
        """Match styles based on image content.

        Args:
            scene_type: Detected scene type
            has_faces: Whether faces were detected
            lighting: Detected lighting condition

        Returns:
            List of recommended styles with reasoning
        """
        matches = []

        # Content-specific recommendations
        if has_faces:
            if lighting == "low_light":
                # Portrait in low light
                matches.append({
                    'style': "Film Noir",
                    'description': "High contrast black and white with dramatic shadows",
                    'score': 9,
                    'reasoning': ["Detected faces in low light", "Dramatic lighting suits noir style"]
                })
            else:
                # Standard portrait
                matches.append({
                    'style': "Portrait",
                    'description': "Optimized for portrait photography with skin tone enhancement",
                    'score': 9,
                    'reasoning': ["Detected faces", "Standard portrait enhancement"]
                })

        if scene_type in ["landscape", "nature"]:
            # Landscape scene
            matches.append({
                'style': "Cinematic Teal & Orange",
                'description': "Hollywood-style color grading with teal shadows and orange highlights",
                'score': 8,
                'reasoning': ["Detected landscape/nature scene", "Popular cinematic look for landscapes"]
            })

            matches.append({
                'style': "Anamorphic",
                'description': "Widescreen cinematic look with enhanced contrast and blue lens flares",
                'score': 7,
                'reasoning': ["Detected landscape/nature scene", "Wide cinematic style suits landscape views"]
            })

        if lighting == "low_light":
            # Low light scene
            if not has_faces and scene_type not in ["landscape", "nature"]:
                matches.append({
                    'style': "Anamorphic",
                    'description': "Widescreen cinematic look with enhanced contrast and blue lens flares",
                    'score': 6,
                    'reasoning': ["Low light scene detected", "Cinematic look enhances mood"]
                })

        if scene_type == "indoor":
            # Indoor scene
            matches.append({
                'style': "Vintage",
                'description': "Classic film-inspired look with faded colors",
                'score': 5,
                'reasoning': ["Detected indoor scene", "Vintage style works well with indoor settings"]
            })

        # Add auto-enhance as a fallback
        if not matches:
            matches.append({
                'style': "Auto-Enhance",
                'description': "Balanced automatic enhancement for most photos",
                'score': 5,
                'reasoning': ["General purpose enhancement"]
            })

        # Sort by score
        matches.sort(key=lambda x: x['score'], reverse=True)
        return matches[:3]  # Return top 3 matches

    def apply_style(self, image: np.ndarray, style_name: str) -> np.ndarray:
        """Apply the specified style to an image.

        Args:
            image: Input image
            style_name: Name of the style to apply

        Returns:
            Processed image
        """
        # Import here to avoid circular imports
        executor = ImageExecutor()

        # Apply the style using the executor
        processed = executor.apply(image, [], style_name)

        return processed


In [ ]:
def display_image(image, title=None):
    """Display an image with an optional title."""
    plt.figure(figsize=(10, 8))
    if isinstance(image, str):
        # If image is a file path, load it
        image = load_image(image)

    plt.imshow(image)
    plt.axis('off')
    if title:
        plt.title(title, fontsize=14)
    plt.show()

def display_before_after(before, after, titles=None):
    """Display before and after images side by side."""
    if titles is None:
        titles = ['Before', 'After']

    plt.figure(figsize=(20, 10))

    plt.subplot(1, 2, 1)
    if isinstance(before, str):
        before = load_image(before)
    plt.imshow(before)
    plt.axis('off')
    plt.title(titles[0], fontsize=14)

    plt.subplot(1, 2, 2)
    if isinstance(after, str):
        after = load_image(after)
    plt.imshow(after)
    plt.axis('off')
    plt.title(titles[1], fontsize=14)

    plt.tight_layout()
    plt.show()

def display_multiple(images, titles=None, cols=3):
    """Display multiple images in a grid."""
    n = len(images)
    rows = (n + cols - 1) // cols

    plt.figure(figsize=(5*cols, 5*rows))

    for i, image in enumerate(images):
        plt.subplot(rows, cols, i+1)
        if isinstance(image, str):
            image = load_image(image)
        plt.imshow(image)
        plt.axis('off')
        if titles and i < len(titles):
            plt.title(titles[i], fontsize=12)

    plt.tight_layout()
    plt.show()


## Load Test Image

Let's load a test image that we'll use throughout this notebook.


In [ ]:
# Load a test image from the web
test_image_url = 'https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg'
image = load_image(test_image_url)
# Note: Using a sample image from OpenCV's GitHub repository (Lena image)

# Display the image
display_image(image, "Test Image")


# Part 1: AI-Powered Image Analysis

One of the key innovations in our photo editor is the ability to analyze image content using AI. This allows the application to understand what's in the photo and make intelligent recommendations based on the content.


In [ ]:
# Initialize the AI image analyzer
ai_analyzer = AIImageAnalyzer()

# Analyze the image
adjustments, analysis = ai_analyzer.analyze(image)

# Display the analysis results
print("AI Image Analysis Results:")
print(f"Scene Type: {analysis['scene_type']}")
print(f"Lighting Condition: {analysis['lighting_condition']}")
print(f"Faces Detected: {analysis['face_count']}")
print("\nDetected Objects:")
for obj in analysis['objects']:
    print(f"- {obj['class']} (confidence: {obj['confidence']:.2f})")

print("\nDominant Colors:")
for i, color in enumerate(analysis['color_palette'][:3]):
    print(f"- Color {i+1}: RGB{tuple(color)}")

print("\nRecommended Adjustments:")
for adj in adjustments:
    print(f"- {adj.parameter}: {adj.suggested} {adj.unit} - {adj.description}")


### Visualizing the AI Analysis

Let's visualize some of the analysis results to better understand what the AI is seeing.


In [ ]:
# Visualize the color palette
def display_color_palette(colors):
    """Display the color palette as color swatches."""
    plt.figure(figsize=(10, 2))
    for i, color in enumerate(colors):
        plt.subplot(1, len(colors), i+1)
        plt.axis('off')
        plt.imshow([[color]])
        plt.title(f"RGB{tuple(color)}")
    plt.tight_layout()
    plt.show()

print("Dominant Color Palette:")
display_color_palette(analysis['color_palette'][:5])


### Applying AI-Recommended Adjustments

Now that we have AI-recommended adjustments, let's apply them to the image and see the results.


In [ ]:
# Apply the recommended adjustments
adjusted_image = apply_adjustments(image, adjustments)

# Display before and after
display_before_after(image, adjusted_image, ["Original Image", "AI-Enhanced Image"])


# Part 2: Natural Language Photo Editing

Another innovative feature of our photo editor is the ability to edit photos using natural language instructions. This allows users to describe what they want in plain English, without needing to understand technical terms or complex editing tools.

## Image Processing Functions

Let's implement the image processing functions that our natural language processor will use:


In [ ]:
def adjust_exposure(image, amount):
    """Adjust image exposure/brightness.

    Args:
        image: Input image
        amount: Adjustment amount (-1.0 to 1.0)

    Returns:
        Adjusted image
    """
    # Simple implementation for demonstration
    result = image.copy().astype(float)
    result = result * (1 + amount)
    return np.clip(result, 0, 255).astype(np.uint8)

def adjust_contrast(image, multiplier):
    """Adjust image contrast.

    Args:
        image: Input image
        multiplier: Contrast multiplier (0.5 to 2.0)

    Returns:
        Adjusted image
    """
    # Simple implementation for demonstration
    mean = np.mean(image, axis=(0, 1))
    result = image.copy().astype(float)
    for i in range(3):
        result[:,:,i] = (result[:,:,i] - mean[i]) * multiplier + mean[i]
    return np.clip(result, 0, 255).astype(np.uint8)

def adjust_saturation(image, adjustment):
    """Adjust image saturation/vibrance.

    Args:
        image: Input image
        adjustment: Saturation adjustment (-1.0 to 1.0)

    Returns:
        Adjusted image
    """
    # Convert to HSV, adjust S channel, convert back to RGB
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV).astype(float)
    hsv[:,:,1] = hsv[:,:,1] * (1 + adjustment)
    hsv[:,:,1] = np.clip(hsv[:,:,1], 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)

def adjust_temperature(image, adjustment):
    """Adjust image color temperature (warmth/coolness).

    Args:
        image: Input image
        adjustment: Temperature adjustment (-0.5 to 0.5)

    Returns:
        Adjusted image
    """
    # Simple implementation - increase red for warmth, blue for coolness
    result = image.copy().astype(float)
    if adjustment > 0:  # Warm
        result[:,:,0] = np.clip(result[:,:,0] * (1 + adjustment), 0, 255)  # Red
        result[:,:,2] = np.clip(result[:,:,2] * (1 - adjustment/2), 0, 255)  # Blue
    else:  # Cool
        result[:,:,2] = np.clip(result[:,:,2] * (1 - adjustment), 0, 255)  # Blue
        result[:,:,0] = np.clip(result[:,:,0] * (1 + adjustment/2), 0, 255)  # Red
    return result.astype(np.uint8)

def adjust_sharpness(image, strength):
    """Adjust image sharpness.

    Args:
        image: Input image
        strength: Sharpness strength (0.0 to 1.0)

    Returns:
        Adjusted image
    """
    # Simple implementation using unsharp masking
    blur = cv2.GaussianBlur(image, (0, 0), 3)
    result = image.copy().astype(float)
    result = result + strength * (image.astype(float) - blur)
    return np.clip(result, 0, 255).astype(np.uint8)

def reduce_noise(image, strength):
    """Reduce noise in the image.

    Args:
        image: Input image
        strength: Noise reduction strength (0.0 to 1.0)

    Returns:
        Adjusted image
    """
    # Simple implementation using bilateral filter
    # Adjust parameters based on strength
    d = int(5 + strength * 10)  # Diameter of each pixel neighborhood
    sigma_color = 50 + strength * 100  # Filter sigma in the color space
    sigma_space = 50 + strength * 100  # Filter sigma in the coordinate space

    return cv2.bilateralFilter(image, d, sigma_color, sigma_space)


## Setting Up the Natural Language Processor

Now let's set up our natural language processor and register the image processing functions we defined above. This will allow the processor to map natural language instructions to specific operations.


In [ ]:
# Initialize the natural language processor
nl_processor = NLProcessor()

# Register our image processing functions
nl_processor.register_function(
    "adjust_exposure",
    adjust_exposure,
    "Adjust the brightness/exposure of the image",
    {"amount": {"type": "number", "description": "Amount to adjust exposure (-1.0 to 1.0)"}}
)

nl_processor.register_function(
    "adjust_contrast",
    adjust_contrast,
    "Adjust the contrast of the image",
    {"multiplier": {"type": "number", "description": "Contrast multiplier (0.5 to 2.0)"}}
)

nl_processor.register_function(
    "adjust_saturation",
    adjust_saturation,
    "Adjust the color saturation/vibrance of the image",
    {"adjustment": {"type": "number", "description": "Saturation adjustment (-1.0 to 1.0)"}}
)

nl_processor.register_function(
    "adjust_temperature",
    adjust_temperature,
    "Adjust the color temperature (warmth/coolness) of the image",
    {"adjustment": {"type": "number", "description": "Temperature adjustment (-0.5 to 0.5)"}}
)

nl_processor.register_function(
    "adjust_sharpness",
    adjust_sharpness,
    "Adjust the sharpness/clarity of the image",
    {"strength": {"type": "number", "description": "Sharpness strength (0.0 to 1.0)"}}
)

nl_processor.register_function(
    "reduce_noise",
    reduce_noise,
    "Reduce noise/grain in the image",
    {"strength": {"type": "number", "description": "Noise reduction strength (0.0 to 1.0)"}}
)


## Processing Natural Language Instructions

Let's try some natural language instructions and see how the system interprets and applies them.


In [ ]:
# Define a function to process and display results
def process_instruction(image, instruction):
    """Process a natural language instruction and display results."""
    print(f"Instruction: '{instruction}'")

    # Process the instruction
    processed_image, metadata = nl_processor.process(image, instruction)

    # Display the functions that were called
    print("\nFunctions called:")
    for func_call in metadata['functions_called']:
        print(f"- {func_call['name']}({', '.join([f'{k}={v}' for k, v in func_call['args'].items()])})")

    # Display before and after
    display_before_after(image, processed_image, ["Original Image", f"After: '{instruction}'"])

    return processed_image

# Try a simple instruction
result1 = process_instruction(image, "Make the image warmer and increase the contrast slightly")


Let's try some more complex instructions to see how the system handles them.


In [ ]:
# Try more complex instructions
instructions = [
    "Make the colors more vibrant and add some warmth",
    "Increase contrast dramatically and make it cooler",
    "Brighten the dark areas and add clarity",
    "Give it a soft, dreamy look with reduced contrast",
    "Sharpen the details and make colors pop"
]

results = []

for instruction in instructions:
    print(f"\n{'='*50}\n")
    result = process_instruction(image, instruction)
    results.append(result)


## Conversational Editing Workflow

One of the most powerful applications of natural language photo editing is the ability to guide users through an iterative editing process, similar to working with a professional photo editor. Let's demonstrate this conversational editing workflow:


In [ ]:
def conversational_editing_workflow(image):
    """Demonstrate a conversational editing workflow."""
    print("=== Conversational Photo Editing Workflow ===\n")
    print("Starting with the original image:")
    display_image(image, "Original Image")

    # Step 1: Initial assessment and basic enhancement
    print("\nStep 1: Initial assessment and basic enhancement")
    print("User: \"Enhance this photo to make it look better overall\"")

    current_image, metadata = nl_processor.process(image, "Enhance this photo to make it look better overall")

    print("\nAI: \"I've made some basic enhancements. I've slightly increased the exposure, added a bit of contrast, and made the colors more vibrant. Here's the result:\"")
    print("\nOperations performed:")
    for func_call in metadata['functions_called']:
        print(f"- {func_call['name']}({', '.join([f'{k}={v}' for k, v in func_call['args'].items()])})")

    display_image(current_image, "After Basic Enhancement")

    # Step 2: Specific adjustment based on user feedback
    print("\nStep 2: Specific adjustment based on user feedback")
    print("User: \"It looks better, but I'd like it to be a bit warmer and more dramatic\"")

    previous_image = current_image.copy()
    current_image, metadata = nl_processor.process(current_image, "Make it warmer and more dramatic")

    print("\nAI: \"I've added warmth by adjusting the color temperature and increased the contrast for a more dramatic look. Here's the updated image:\"")
    print("\nOperations performed:")
    for func_call in metadata['functions_called']:
        print(f"- {func_call['name']}({', '.join([f'{k}={v}' for k, v in func_call['args'].items()])})")

    display_before_after(previous_image, current_image, ["After Basic Enhancement", "Warmer and More Dramatic"])

    # Step 3: Fine-tuning
    print("\nStep 3: Fine-tuning")
    print("User: \"That's closer to what I want, but now the colors are a bit too intense. Can you tone down the saturation slightly but keep the contrast?\"")

    previous_image = current_image.copy()
    current_image, metadata = nl_processor.process(current_image, "Reduce saturation slightly but maintain contrast")

    print("\nAI: \"I've reduced the color saturation while maintaining the contrast levels. Here's the result:\"")
    print("\nOperations performed:")
    for func_call in metadata['functions_called']:
        print(f"- {func_call['name']}({', '.join([f'{k}={v}' for k, v in func_call['args'].items()])})")

    display_before_after(previous_image, current_image, ["Warmer and More Dramatic", "Fine-tuned"])

    # Step 4: Final touches
    print("\nStep 4: Final touches")
    print("User: \"That's looking good! As a final touch, can you sharpen it a bit to bring out the details?\"")

    previous_image = current_image.copy()
    current_image, metadata = nl_processor.process(current_image, "Sharpen to bring out details")

    print("\nAI: \"I've applied sharpening to enhance the details. Here's your final image:\"")
    print("\nOperations performed:")
    for func_call in metadata['functions_called']:
        print(f"- {func_call['name']}({', '.join([f'{k}={v}' for k, v in func_call['args'].items()])})")

    display_before_after(previous_image, current_image, ["Fine-tuned", "Final Image"])

    # Show the complete transformation
    print("\nComplete Transformation:")
    display_before_after(image, current_image, ["Original Image", "Final Edited Image"])

    return current_image

# Run the conversational editing workflow
final_image = conversational_editing_workflow(image)


# Part 3: RAG-Based Style Recommendations

Our photo editor also uses Retrieval Augmented Generation (RAG) to recommend cinematic styles based on image content. This combines a knowledge base of cinematography techniques with AI image analysis to suggest styles that match the content of the photo.

## Understanding Retrieval Augmented Generation (RAG) for Style Recommendations

Retrieval Augmented Generation (RAG) is a powerful AI technique that combines the strengths of retrieval-based systems with generative models. In our photo editing application, RAG works by:

1. **Retrieving relevant information** from a knowledge base of cinematography styles and techniques
2. **Augmenting the recommendation process** with this retrieved knowledge
3. **Generating style recommendations** that are tailored to the specific image content

This approach has several advantages over traditional preset filters:

- **Content-aware recommendations**: Styles are suggested based on what's in the photo
- **Educational value**: Users learn about cinematography techniques and why they work
- **Flexibility**: The system can adapt to both image content and user descriptions
- **Transparency**: Clear explanations for why each style is recommended


In [ ]:
# Initialize the RAG style engine
rag_engine = RAGStyleEngine()

# Get style recommendations based on image content
recommendations = rag_engine.recommend_style(image)

# Display the recommendations
print("Style Recommendations Based on Image Content:")
for i, rec in enumerate(recommendations):
    print(f"\n{i+1}. {rec['style']} (Score: {rec['score']})")
    print(f"   Description: {rec['description']}")
    print(f"   Reasoning:")
    for reason in rec['reasoning']:
        print(f"   - {reason}")


Now, let's apply these recommended styles to our image and see the results.


In [ ]:
# Apply the recommended styles
styled_images = []
style_names = []

for rec in recommendations:
    style_name = rec['style']
    styled = rag_engine.apply_style(image, style_name)
    styled_images.append(styled)
    style_names.append(style_name)

# Display the original and styled images
display_multiple([image] + styled_images, ["Original"] + style_names)


## Style Recommendations Based on Description

We can also recommend styles based on a description provided by the user. This allows users to describe the look they want in natural language, and the system will find matching styles.


In [ ]:
# Get style recommendations based on a description
description = "I want a dramatic movie look with high contrast"
desc_recommendations = rag_engine.recommend_style(image, description)

# Display the recommendations
print(f"Style Recommendations Based on Description: '{description}'")
for i, rec in enumerate(desc_recommendations):
    print(f"\n{i+1}. {rec['style']} (Score: {rec['score']})")
    print(f"   Description: {rec['description']}")
    print(f"   Reasoning:")
    for reason in rec['reasoning']:
        print(f"   - {reason}")


Let's apply these description-based style recommendations.


In [ ]:
# Apply the description-based recommended styles
desc_styled_images = []
desc_style_names = []

for rec in desc_recommendations:
    style_name = rec['style']
    styled = rag_engine.apply_style(image, style_name)
    desc_styled_images.append(styled)
    desc_style_names.append(style_name)

# Display the original and styled images
display_multiple([image] + desc_styled_images, ["Original"] + desc_style_names)


## Style-Based Storytelling

One of the most powerful applications of RAG-based style recommendations is helping users tell visual stories through consistent styling. Different scenes in a story might require different cinematic looks to convey the right mood and atmosphere.


In [ ]:
def style_based_storytelling():
    """Demonstrate style-based storytelling with RAG recommendations."""
    print("=== Style-Based Storytelling ===\n")
    print("Imagine you're creating a visual story with different scenes. Each scene needs a different mood and atmosphere.")

    # Define our story scenes
    story_scenes = [
        {
            "scene_name": "Opening Scene - Mysterious Beginning",
            "description": "The story begins with a mysterious, moody atmosphere",
            "style_query": "mysterious dark moody atmosphere like a thriller"
        },
        {
            "scene_name": "Flashback Scene - Nostalgic Past",
            "description": "We flash back to happier times in the past",
            "style_query": "warm nostalgic vintage look"
        },
        {
            "scene_name": "Action Scene - Dramatic Confrontation",
            "description": "The protagonist faces a dramatic confrontation",
            "style_query": "dramatic high contrast action movie style"
        },
        {
            "scene_name": "Resolution Scene - Hopeful Ending",
            "description": "The story resolves with a hopeful, uplifting ending",
            "style_query": "bright vibrant hopeful cinematic"
        }
    ]

    # Process each scene
    for scene in story_scenes:
        print(f"\n{'='*80}\n{scene['scene_name']}\n{'='*80}")
        print(f"Scene Description: {scene['description']}")
        print(f"Style Query: \"{scene['style_query']}\"")

        # Get style recommendations for this scene
        recommendations = rag_engine.recommend_style(image, scene['style_query'])

        # Display the top recommendation
        if recommendations:
            top_rec = recommendations[0]
            print(f"\nRecommended Style: {top_rec['style']} (Score: {top_rec['score']})")
            print(f"Style Description: {top_rec['description']}")
            print("Reasoning:")
            for reason in top_rec['reasoning']:
                print(f"- {reason}")

            # Apply the style
            styled_image = rag_engine.apply_style(image, top_rec['style'])

            # Display the result
            display_before_after(image, styled_image, 
                               ["Original Image", f"{scene['scene_name']} with '{top_rec['style']}' style"])

# Run the storytelling demonstration
style_based_storytelling()


# Part 4: Innovative Use Case - AI-Guided Creative Photography

Now let's explore an innovative use case that combines all of these AI capabilities: AI-guided creative photography. In this scenario, the AI analyzes an image, suggests creative directions, and helps the user achieve a specific artistic vision.

This approach is particularly valuable for:
- Amateur photographers looking to achieve professional results
- Creative professionals seeking inspiration
- Educators teaching photography and editing techniques

Let's see how this works in practice.


In [ ]:
def ai_guided_creative_workflow(image, creative_direction):
    """Demonstrate an AI-guided creative workflow.

    Args:
        image: Input image
        creative_direction: Description of the desired creative direction

    Returns:
        Tuple of (final image, workflow steps)
    """
    workflow_steps = []
    results = [image.copy()]

    # Step 1: Analyze the image
    ai_analyzer = AIImageAnalyzer()
    adjustments, analysis = ai_analyzer.analyze(image)

    workflow_steps.append({
        "step": "Image Analysis",
        "description": f"AI analyzed the image and identified it as a {analysis['scene_type']} scene with {analysis['lighting_condition']} lighting."
    })

    # Step 2: Apply basic adjustments
    basic_adjusted = apply_adjustments(image, adjustments)
    results.append(basic_adjusted)

    workflow_steps.append({
        "step": "Basic Adjustments",
        "description": f"Applied {len(adjustments)} AI-recommended adjustments to optimize the image."
    })

    # Step 3: Find styles matching the creative direction
    rag_engine = RAGStyleEngine()
    style_recs = rag_engine.recommend_style(image, creative_direction)

    if style_recs:
        # Apply the top recommended style
        top_style = style_recs[0]['style']
        styled_image = rag_engine.apply_style(basic_adjusted, top_style)
        results.append(styled_image)

        workflow_steps.append({
            "step": "Style Application",
            "description": f"Applied '{top_style}' style based on the creative direction: '{creative_direction}'"
        })

    # Step 4: Fine-tune with natural language processing
    nl_processor = NLProcessor()

    # Register the same functions as before
    nl_processor.register_function("adjust_exposure", adjust_exposure, 
                                 "Adjust the brightness/exposure of the image",
                                 {"amount": {"type": "number", "description": "Amount to adjust exposure (-1.0 to 1.0)"}})

    nl_processor.register_function("adjust_contrast", adjust_contrast,
                                 "Adjust the contrast of the image",
                                 {"multiplier": {"type": "number", "description": "Contrast multiplier (0.5 to 2.0)"}})

    nl_processor.register_function("adjust_saturation", adjust_saturation,
                                 "Adjust the color saturation of the image",
                                 {"adjustment": {"type": "number", "description": "Saturation adjustment (-1.0 to 1.0)"}})

    nl_processor.register_function("adjust_temperature", adjust_temperature,
                                 "Adjust the color temperature (warmth/coolness) of the image",
                                 {"adjustment": {"type": "number", "description": "Temperature adjustment (-0.5 to 0.5)"}})

    nl_processor.register_function("adjust_sharpness", adjust_sharpness,
                                 "Adjust the sharpness/clarity of the image",
                                 {"strength": {"type": "number", "description": "Sharpness strength (0.0 to 1.0)"}})

    nl_processor.register_function("reduce_noise", reduce_noise,
                                 "Reduce noise/grain in the image",
                                 {"strength": {"type": "number", "description": "Noise reduction strength (0.0 to 1.0)"}})

    # Generate a fine-tuning instruction based on the creative direction
    fine_tuning_instruction = f"Fine-tune for {creative_direction}"
    final_image, metadata = nl_processor.process(results[-1], fine_tuning_instruction)
    results.append(final_image)

    workflow_steps.append({
        "step": "Fine-tuning",
        "description": f"Applied natural language fine-tuning: '{fine_tuning_instruction}'",
        "functions": [f"{func['name']}({', '.join([f'{k}={v}' for k, v in func['args'].items()])})" 
                     for func in metadata['functions_called']]
    })

    return final_image, results, workflow_steps


In [ ]:
# Try the AI-guided creative workflow with different creative directions
creative_directions = [
    "dramatic cinematic look",
    "warm vintage feel",
    "professional portrait style"
]

for direction in creative_directions:
    print(f"\n\n=== AI-Guided Creative Workflow: '{direction}' ===\n")

    final_image, workflow_images, steps = ai_guided_creative_workflow(image, direction)

    # Display the workflow steps
    for i, step in enumerate(steps):
        print(f"\nStep {i+1}: {step['step']}")
        print(f"  {step['description']}")
        if 'functions' in step:
            print("  Functions applied:")
            for func in step['functions']:
                print(f"  - {func}")

    # Display the workflow images
    step_titles = ["Original"] + [step["step"] for step in steps]
    display_multiple(workflow_images, step_titles)


# Conclusion

In this comprehensive notebook, we've demonstrated how generative AI transforms the photo editing experience by making it more accessible, intelligent, and efficient. We've explored three key innovations:

1. **AI-powered image analysis** that understands content and recommends appropriate adjustments
2. **Natural language photo editing** that allows users to describe what they want in plain English
3. **RAG-based style recommendations** that suggest cinematic styles based on image content and user descriptions

We've also shown how these capabilities can be combined in an **AI-guided creative workflow** that helps users achieve specific artistic visions.

These AI-powered approaches democratize photo editing by removing the technical barriers that traditionally made it difficult for casual photographers to achieve professional-looking results. By understanding what's in the photo and what the user wants to achieve, the AI can guide them through the editing process and help them create images that match their creative vision.

Key benefits include:

- **Accessibility**: No technical knowledge required to achieve professional results
- **Efficiency**: Faster editing with fewer steps and trial-and-error
- **Intuitiveness**: Edit using familiar language instead of technical controls
- **Education**: Learn about cinematography techniques and visual aesthetics
- **Personalization**: Get recommendations tailored to specific image content

The future of photo editing is not about replacing human creativity, but about augmenting it with AI that understands both the technical aspects of photography and the artistic intentions of the user.
